# **Data Scraping and Exploration with Pandas by Analysing the Post-University Salaries of Graduates by Major**

---
### We will initially gather data from [Payscale](https://www.payscale.com/college-salary-report/majors-that-pay-you-back/bachelors?orderBy=percentHighMeaning&ascending=false) and parse it using beautiful soup
---
### Following this we try to explore data with Pandas by Analysing the Post-University Salaries of Graduates 





## This is the script which will scrap the web to collect live data available on Payscale, and it will parse it and get the important data components with the help of BeautifulSoup.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

URL_PAYSCALE = 'https://www.payscale.com/college-salary-report/majors-that-pay-you-back/bachelors/page/'

page = 1

# Declare lists with all the data from all the pages
total_ranks_list = []
total_majors_list = []
total_early_career_pay_list = []
total_mid_career_pay_list = []
total_high_meaning_list = []

# Will scrap 34 pages 
while page <= 34:

    response = requests.get(url=f"{URL_PAYSCALE}{page}")
    web_page_data = response.text

    soup = BeautifulSoup(web_page_data, 'html.parser')

    # Index
    ranks = soup.find_all(name='td', class_='csr-col--rank')
    # Majors
    majors = soup.find_all(name='td', class_='csr-col--school-name')
    # pays
    career_pays = soup.find_all(name='td', class_='csr-col--right')

    # Lists for each page to add to the total lists
    ranks_list = [rank.getText().split(':')[1] for rank in ranks]
    majors_list = [major.getText().split(':')[1] for major in majors]

    early_career_pay_list = []
    mid_career_pay_list = []
    high_meaning_list = []

    # Use for loop because it's easier to read at this case
    for pay in career_pays:
        if pay.getText().split(':')[0] == 'Early Career Pay':
            early_career_pay_list.append(pay.getText().split(':')[1])
        elif pay.getText().split(':')[0] == 'Mid-Career Pay':
            mid_career_pay_list.append(pay.getText().split(':')[1])
        elif pay.getText().split(':')[0] == '% High Meaning':
            high_meaning_list.append(pay.getText().split(':')[1])

    total_ranks_list.append(ranks_list)
    total_majors_list.append(majors_list)
    total_early_career_pay_list.append(early_career_pay_list)
    total_mid_career_pay_list.append(mid_career_pay_list)
    total_high_meaning_list.append(high_meaning_list)

    page += 1

list_to_csv = []

for i in range(33):
    for n in range(25):
        data_to_csv = []
        data_to_csv.append(total_ranks_list[i][n])
        data_to_csv.append(total_majors_list[i][n])
        # Convert all pay values with $ sign from string to float dtype
        data_to_csv.append(float(total_early_career_pay_list[i][n].split('$')[1].replace(',', '')))
        data_to_csv.append(float(total_mid_career_pay_list[i][n].split('$')[1].replace(',', '')))
        data_to_csv.append(total_high_meaning_list[i][n])

        list_to_csv.append(data_to_csv)

data = pd.DataFrame(list_to_csv,
                    columns=['Rank', 'Major', 'Early Career Pay', 'Mid-Career Pay', 'Meaning %'])

## Now that we have data we will analyze it using pandas

Basic format of data.

In [ ]:
data.head()

,Rank,Major,Early Career Pay,Mid-Career Pay,Meaning %
0,1,Petroleum Engineering,93200.0,187300.0,67%
1,2,Operations Research & Industrial Engineering,84800.0,170400.0,28%
2,3,Electrical Engineering & Computer Science (EECS),108500.0,159300.0,46%
3,4,Interaction Design,68300.0,155800.0,55%
4,5,Public Accounting,59800.0,147700.0,47%


Volume of the data and type

In [ ]:
print(data.shape)
print(data.info())

(825, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 825 entries, 0 to 824
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              825 non-null    object 
 1   Major             825 non-null    object 
 2   Early Career Pay  825 non-null    float64
 3   Mid-Career Pay    825 non-null    float64
 4   Meaning %         825 non-null    object 
dtypes: float64(2), object(3)
memory usage: 71.0+ KB
None


In [ ]:
data.describe()

,Early Career Pay,Mid-Career Pay
count,825.000000,825.000000
mean,52729.818182,87665.818182
std,10420.859080,21294.068984
min,34500.000000,45000.000000
25%,45100.000000,72100.000000
50%,50200.000000,85200.000000
75%,59100.000000,100900.000000
max,108500.000000,187300.000000


Dropping all the null values

In [ ]:
data = data.dropna()

In [ ]:
data.shape

(825, 5)

### Top 5 majors careers with the highest early career pay :

In [ ]:
top_early_payer = data.sort_values("Early Career Pay", ascending=False).head()
top_early_payer[["Major", "Early Career Pay"]]

,Major,Early Career Pay
2,Electrical Engineering & Computer Science (EECS),108500.0
75,Physician Assistant Studies,95900.0
0,Petroleum Engineering,93200.0
1,Operations Research & Industrial Engineering,84800.0
158,Nuclear Engineering Technology (NET),83500.0


### Details of career with the highest early career pay.

In [ ]:
data.loc[data["Early Career Pay"].idxmax()]

Rank                                                               3
Major               Electrical Engineering & Computer Science (EECS)
Early Career Pay                                            108500.0
Mid-Career Pay                                              159300.0
Meaning %                                                        46%
Name: 2, dtype: object

### Details of career with the lowest early career pay.

In [ ]:
data.loc[data["Early Career Pay"].idxmin()]

Rank                          805
Major               Voice & Opera
Early Career Pay          34500.0
Mid-Career Pay            53300.0
Meaning %                     57%
Name: 804, dtype: object

### Top 5 majors careers with the highest mid career pay(after 10 years of experience : )

In [ ]:
top_early_payer = data.sort_values("Mid-Career Pay", ascending=False).head()
top_early_payer[["Major", "Mid-Career Pay"]]

,Major,Mid-Career Pay
0,Petroleum Engineering,187300.0
1,Operations Research & Industrial Engineering,170400.0
2,Electrical Engineering & Computer Science (EECS),159300.0
3,Interaction Design,155800.0
4,Public Accounting,147700.0


### Details of career with highest Mid-Career Pay.

In [ ]:
data.loc[data["Mid-Career Pay"].idxmax()]

Rank                                    1
Major               Petroleum Engineering
Early Career Pay                  93200.0
Mid-Career Pay                   187300.0
Meaning %                             67%
Name: 0, dtype: object

### Details of career with lowest Mid-Career Pay.

In [ ]:
data.loc[data["Mid-Career Pay"].idxmin()]

Rank                          825
Major               Mental Health
Early Career Pay          36900.0
Mid-Career Pay            45000.0
Meaning %                       -
Name: 824, dtype: object

### Adding another fields spread which is difference between mid-career pay and early career pay.

In [ ]:
data.insert(1, 'Spread', data['Mid-Career Pay'] - data['Early Career Pay'])
data.head()

,Rank,Spread,Major,Early Career Pay,Mid-Career Pay,Meaning %
0,1,94100.0,Petroleum Engineering,93200.0,187300.0,67%
1,2,85600.0,Operations Research & Industrial Engineering,84800.0,170400.0,28%
2,3,50800.0,Electrical Engineering & Computer Science (EECS),108500.0,159300.0,46%
3,4,87500.0,Interaction Design,68300.0,155800.0,55%
4,5,87900.0,Public Accounting,59800.0,147700.0,47%


## Relation years of experience and pay

### Lowest increment of salary (No significant change even after 10 years of experience )

In [ ]:
# Lowest increment of salary (No significant change even after 10 years of experience  )
car_lowest_jump = data.sort_values("Spread")
car_lowest_jump.head()

,Rank,Spread,Major,Early Career Pay,Mid-Career Pay,Meaning %
821,822,7200.0,Rehabilitation Counseling,39200.0,46400.0,-
808,808,7400.0,Human Services Management,45600.0,53000.0,-
793,794,7900.0,History Teacher Education,47700.0,55600.0,46%
824,825,8100.0,Mental Health,36900.0,45000.0,-
806,806,8800.0,Conflict Resolution,44400.0,53200.0,50%


### Career with the highest possibilities of increment of salary with amount of work experience (No significant change even after 10 years of experience )

In [ ]:
car_highest_jump = data.sort_values("Spread", ascending=False)
car_highest_jump.head()

,Rank,Spread,Major,Early Career Pay,Mid-Career Pay,Meaning %
0,1,94100.0,Petroleum Engineering,93200.0,187300.0,67%
4,5,87900.0,Public Accounting,59800.0,147700.0,47%
3,4,87500.0,Interaction Design,68300.0,155800.0,55%
1,2,85600.0,Operations Research & Industrial Engineering,84800.0,170400.0,28%
10,11,82300.0,Information & Computer Science,58600.0,140900.0,62%


## Perception of societal impact amongst alumni

### Alumni who say their work makes the world a better place and work they do is meaningful and has positive impact on the world and society

Top 5 Career with the highest positive rate

In [ ]:
# alumni who say their work makes the world a better place and work they do is meaningful and has positive impact on the world
car_highest_meaning = data.sort_values("Meaning %", ascending=False)
car_highest_meaning[["Major", "Meaning %"]].head()

,Major,Meaning %
754,Music Therapy,93%
129,Medicine,92%
594,Cardiopulmonary Science,90%
295,Radiation Therapy,90%
520,Cytotechnology,89%



5 Career with the lowest positive rate


In [ ]:
car_lowest_meaning = data.where(data['Meaning %'] != '-').sort_values("Meaning %")
car_lowest_meaning[["Major", "Meaning %"]].head()

,Major,Meaning %
102,Japanese Studies,21%
189,International Marketing,22%
634,Commercial Photography,23%
188,Food Marketing,24%
653,Web Design,25%
